*DISEASE CLASSIFICATION TIME-SERIES* MODEL

In [ ]:
# imports
import datetime
from datetime import date
import pandas as pd
import numpy as np
from plotly import __version__
%matplotlib inline

import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 


cf.go_offline()

Uploading CSV file

In [ ]:
# File named in ./data.csv
# NOTE: Please "Allow 3rd Party Cookies" in Chrome Options

from google.colab import files
uploaded = files.upload()

In [ ]:
print(uploaded)

In [ ]:
import pandas as pd
import io
  
df = pd.read_csv(io.BytesIO(uploaded['dataset_min.csv']))
df_size = len(df.index)

df["Index"] = np.linspace(start = 0, stop = df_size-1, num = df_size, dtype = int)

In [ ]:
# visualizing

print(df.head().to_string())

print(df[["RF", "MaxT"]])

print(df["RF"][0])

print(type(df['RF']))

print(type(df['RF'].to_numpy()))

Visualizing Data

In [ ]:
# Creating Simple Dynamic Graph to see all data
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

fig = plt.figure(figsize=(23, 6), dpi=80)
ax = plt.axes()

# NOTE : Chane the value of var for any other header to get different results
var = "RF"
plt.plot(df['Date'].to_numpy(), df[var].to_numpy());

In [ ]:
init_notebook_mode(connected=False)

In [ ]:
# Create function for Colab
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
  '''))

In [ ]:
configure_plotly_browser_state()
df.iplot(kind='line', x='Index', y=['RF'], color=['white'], 
theme='solar', mode='markers',title='Annual Rainfall in the city Peachtree City, GA')
plt.show()

In [ ]:
configure_plotly_browser_state()
df.iplot(kind='line',x='Index',y=['MinT', "MaxT"], color=['white', 'gold'], 
theme='solar', mode='markers',title='Annual Rainfall in the city Peachtree City, GA')
plt.show()

Pre Processing Data

In [ ]:
# preproceessing data
def one_hot_prob_dist(val):
  if val == 0 :
    return [1, 0, 0, 0]
  elif val == 1 :
    return [0, 1, 0, 0]
  elif val == 2 :
    return [0, 0, 1, 0]
  elif val == 3 :
    return [0, 0, 0, 1]
  else :
    print(val)
    raise ValueError

def create_timesteps(X, y, length, step, n_features):
  # Create Timestep Data
  X = X.reshape(length, 1, n_features)

  # Num samples = length - step + 1
  samples = length - step + 1
    
  if step > 1 :
    y = y[step-1:] 

    temp = np.empty(shape=[samples, step, n_features])
    for i in range(samples):
      temp[i] = X[i : i+step].reshape(1, step, n_features)
    return temp, y
  
  return X, y

def prepare_data(data, length, step, n_features):
  # Dividing X and y
  X = data[["RF", "MaxT", "MinT", "RH-I", "RH-II", "C1", "C2", "SS", "WD1", "WD2"]]
  y_temp = data["Disease"]
  y = []

  # print("X & y : ", "\n", X, "\n", y_temp)

  # Create Numpy arrays
  X = X.to_numpy()
  y_temp = y_temp.to_numpy()

  for i in range(len(y_temp)):
    arr = one_hot_prob_dist(y_temp[i])
    y.append(arr)

  y = np.array(y)

  # print("X & y (in numpy) : ", "\n", X, "\n", y)
  # print("X & y (shape) : ", X.shape, ", ", y.shape)

  # Normalizing values
  X = (X - X.min(0)) / X.ptp(0)
  # y = (y - y.min(0)) / y.ptp(0)

  # print("X & y (normalized) : ", "\n", X, "\n", y)

  # reshaping data into 3D structure [example, timesteps, features]
  X, y = create_timesteps(X, y, length, step, n_features)
  
  return X, y

def split_data(X, y, ratio=0.98):
  # Create X_test, X_train, y_test, y_train
  if ratio > 1:
    raise Error
  else :
    tot = X.shape[0]
    div = round(tot*ratio)

    # splitting
    X_train = X[:div, :, :]
    y_train = y[:div]

    X_test = X[div:, :, :]
    y_test = y[div:]
  
  return X_train, y_train, X_test, y_test

In [ ]:
# defining data
step = 15
length = len(df.index)
n_features = 10

X, y = prepare_data(df, length, step, n_features)
length = y.shape[0]

print("X : \n", X.shape, "\n\ny : ", y.shape)


In [ ]:
np.set_printoptions(precision=3)
print("X : \n", X[:6], "\n\ny : ", y[:6])

In [ ]:
X_train, y_train, X_test, y_test = split_data(X, y, ratio=0.8)

In [ ]:
print("X_train : ", X_train[:10], "\n\ny_test : ", y_train[:-10])
print("\n\nX_train and y_test (shape) : ", X_train.shape, ", ", y_test.shape)

Training Models

In [ ]:
# impots
import tensorflow as tf
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import RNN
from tensorflow.keras.layers import Dense

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.framework import ops
from tensorflow.python.training import optimizer
import tensorflow as tf

Optimizer Class

In [ ]:
# Base Optimizer
opti = Adam(learning_rate=0.00005)

baisc functions in optimizer class

```
def _create_slots(self, var_list):
     # Create slots for allocation and later management of additional 
     # variables associated with the variables to train.
     # for example: the first and second moments.
     '''
     for v in var_list:
     self._zeros_slot(v, "m", self._name)
     self._zeros_slot(v, "v", self._name)
     '''
 def _apply_dense(self, grad, var):
     #define your favourite variable update
     # for example:
     '''
     # Here we apply gradient descents by substracting the variables 
     # with the gradient times the learning_rate (defined in __init__)
     var_update = state_ops.assign_sub(var, self.learning_rate * grad) 
     '''
     #The trick is now to pass the Ops in the control_flow_ops and 
     # eventually groups any particular computation of the slots your 
     # wish to keep track of:
     # for example:    
     '''
     m_t = ...m... #do something with m and grad
     v_t = ...v... # do something with v and grad
     '''
  return control_flow_ops.group(*[var_update, m_t, v_t])
```

In [ ]:
class PowerSign(optimizer.Optimizer):
    """Implementation of PowerSign.
    See [Bello et. al., 2017](https://arxiv.org/abs/1709.07417)
    @@__init__
    """
    def __init__(self, learning_rate=0.001,alpha=0.01,beta=0.5, use_locking=False, name="PowerSign"):
        super(PowerSign, self).__init__(use_locking, name)
        self._lr = learning_rate
        self._alpha = alpha
        self._beta = beta
        
        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None
        self._alpha_t = None
        self._beta_t = None

    def _prepare(self):
        self._lr_t = ops.convert_to_tensor(self._lr, name="learning_rate")
        self._alpha_t = ops.convert_to_tensor(self._beta, name="alpha_t")
        self._beta_t = ops.convert_to_tensor(self._beta, name="beta_t")

    def _create_slots(self, var_list):
        # Create slots for the first and second moments.
        for v in var_list:
            self._zeros_slot(v, "m", self._name)

    def _apply_dense(self, grad, var):
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)
        alpha_t = math_ops.cast(self._alpha_t, var.dtype.base_dtype)
        beta_t = math_ops.cast(self._beta_t, var.dtype.base_dtype)

        eps = 1e-7 #cap for moving average
        
        m = self.get_slot(var, "m")
        m_t = m.assign(tf.maximum(beta_t * m + eps, tf.abs(grad)))

        var_update = state_ops.assign_sub(var, lr_t*grad*tf.exp( tf.log(alpha_t)*tf.sign(grad)*tf.sign(m_t))) #Update 'ref' by subtracting 'value
        #Create an op that groups multiple operations.
        #When this op finishes, all ops in input have finished
        return control_flow_ops.group(*[var_update, m_t])
    
    def _apply_sparse(self, grad, var):
        raise NotImplementedError("Sparse gradient updates are not supported.")

In [ ]:
class AddSign(optimizer.Optimizer):
    """Implementation of AddSign.
    See [Bello et. al., 2017](https://arxiv.org/abs/1709.07417)
    @@__init__
    """

    def __init__(self, learning_rate=1.001,alpha=0.01,beta=0.5, use_locking=False, name="AddSign"):
        super(AddSign, self).__init__(use_locking, name)
        self._lr = learning_rate
        self._alpha = alpha
        self._beta = beta
        
        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None
        self._alpha_t = None
        self._beta_t = None
      
    def _prepare(self):
        self._lr_t = ops.convert_to_tensor(self._lr, name="learning_rate")
        self._alpha_t = ops.convert_to_tensor(self._beta, name="beta_t")
        self._beta_t = ops.convert_to_tensor(self._beta, name="beta_t")

    def _create_slots(self, var_list):
        # Create slots for the first and second moments.
        for v in var_list:
            self._zeros_slot(v, "m", self._name)

    def _apply_dense(self, grad, var):
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)
        beta_t = math_ops.cast(self._beta_t, var.dtype.base_dtype)
        alpha_t = math_ops.cast(self._alpha_t, var.dtype.base_dtype)
    
        eps = 1e-7 #cap for moving average
        
        m = self.get_slot(var, "m")
        m_t = m.assign(tf.maximum(beta_t * m + eps, tf.abs(grad)))
        
        var_update = state_ops.assign_sub(var, lr_t*grad*(1.0+alpha_t*tf.sign(grad)*tf.sign(m_t) ) )
        #Create an op that groups multiple operations
        #When this op finishes, all ops in input have finished
        return control_flow_ops.group(*[var_update, m_t])

    def _apply_sparse(self, grad, var):
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)
        beta_t = math_ops.cast(self._beta_t, var.dtype.base_dtype)
        alpha_t = math_ops.cast(self._alpha_t, var.dtype.base_dtype)
    
        eps = 1e-7 #cap for moving average
        
        m = self.get_slot(var, "m")
        m_t = m.assign(tf.maximum(beta_t * m + eps, tf.abs(grad)))
        
        var_update = state_ops.assign_sub(var, lr_t*grad*(1.0+alpha_t*tf.sign(grad)*tf.sign(m_t) ) )
        #Create an op that groups multiple operations
        #When this op finishes, all ops in input have finished
        return control_flow_ops.group(*[var_update, m_t])

    def _resource_apply_dense(self, grad, var):
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)
        beta_t = math_ops.cast(self._beta_t, var.dtype.base_dtype)
        alpha_t = math_ops.cast(self._alpha_t, var.dtype.base_dtype)
    
        eps = 1e-7 #cap for moving average
        
        m = self.get_slot(var, "m")
        m_t = m.assign(tf.maximum(beta_t * m + eps, tf.abs(grad)))
        
        var_update = state_ops.assign_sub(var, lr_t*grad*(1.0+alpha_t*tf.sign(grad)*tf.sign(m_t) ) )
        #Create an op that groups multiple operations
        #When this op finishes, all ops in input have finished
        return control_flow_ops.group(*[var_update, m_t])

In [ ]:
def RosenbrockOpt(optimizer,MAX_EPOCHS = 4000, MAX_STEP = 100):
  '''
	returns distance of each step*MAX_STEP w.r.t minimum (1,1)
	'''

  x1_data = tf.Variable(initial_value=tf.random_uniform([1], minval=-3, maxval=3,seed=0),name='x1')
  x2_data = tf.Variable(initial_value=tf.random_uniform([1], minval=-3, maxval=3,seed=1), name='x2')

  y = tf.add(tf.pow(tf.subtract(1.0, x1_data), 2.0),
  tf.multiply(100.0, tf.pow(tf.subtract(x2_data, tf.pow(x1_data, 2.0)), 2.0)), 'y')

  global_step_tensor = tf.Variable(0, trainable=False, name='global_step')

  train = optimizer.minimize(y,global_step=global_step_tensor)

  sess = tf.Session()

  init = tf.global_variables_initializer()#tf.initialize_all_variables()
  sess.run(init)

  minx = 1.0
  miny = 1.0

  distance = []
  xx_ = sess.run(x1_data)
  yy_ = sess.run(x2_data)
  print(0,xx_,yy_,np.sqrt((minx-xx_)**2+(miny-yy_)**2))

  for step in range(MAX_EPOCHS):
    _, xx_, yy_, zz_ = sess.run([train,x1_data,x2_data,y])
    if step % MAX_STEP == 0:
      print(step+1, xx_,yy_, zz_)
      distance += [ np.sqrt((minx-xx_)**2+(miny-yy_)**2)]
    sess.close()
    return distance

In [ ]:
# New Optimizer
opti = AddSign(learning_rate=0.00005)

Vanilla LSTM

In [ ]:
# defining model
model = Sequential()

# model structure
model.add(LSTM(30, activation='relu', input_shape=(step, n_features)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(4, activation='softmax'))

# compile model
model.compile(optimizer=opti, loss='categorical_crossentropy', metrics=['accuracy'])

# model sumary
print(model.summary())

# train model
epochs = 100
history = model.fit(X_train, y_train, epochs=epochs, verbose=1)

# validation_data=(X_test, y_test), batch_size=20,

Predicting Data and Seeing results using Test Set 

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
num_tests = 10
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:num_tests])
print("predictions shape:", predictions.shape)

In [ ]:
for i in range(num_tests):
  print("Test Value :", y_test[i])
  print("Predicted Value :", predictions[i])
  print("\n")

Visualizing Ouput

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Add data
# check for overfitting and underfitting
loss = history.history['loss']
acc = history.history['accuracy']
epoch = np.arange(epochs) + 1

# Note that even in the OO-style, we use `.pyplot.figure` to create the figure.
fig, ax = plt.subplots()  # Create a figure and an axes.
ax.plot(loss, epoch, label='loss')  # Plot some data on the axes.
ax.plot(acc, epoch, label='accuracy')  # Plot more data on the axes...

ax.set_xlabel('Epochs')  # Add an x-label to the axes.
ax.set_ylabel('Score')  # Add a y-label to the axes.
ax.set_title("Simple Plot")  # Add a title to the axes.
ax.legend()  # Add a legend.

